In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [6]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, msubs
from IPython.display import Image
from IPython.core.display import HTML
import scipy.integrate
import math
from numpy.linalg import matrix_power
from scipy.linalg import expm

%matplotlib notebook

In [7]:
def discretize_ss(A, B, dt):
    A_B = np.block([[A, B],
                [np.zeros((B.shape[1], A.shape[0])), np.zeros((B.shape[1], B.shape[1]))]])
    #print("A shape:", A.shape)
    #print("B shape:", B.shape)
    #print("A_B:", A_B)

    eAt_d = scipy.linalg.expm(A_B * dt)

    A_d_temp = eAt_d[:A.shape[0], :A.shape[0]]

    B_d_temp = eAt_d[:B.shape[0], A.shape[0]:]

    return (A_d_temp, B_d_temp)

In [8]:
def get_integrated_position_trajectory(initial_pos, vel_desired):
    x_ref_temp = np.zeros((n, N))

    pos_desired = initial_pos
    for i in range(0, N):
        pos_desired += vel_desired * dt
        x_ref_temp[:, i] = np.array([0, 0, 0, 0, pos_desired, 1, 0, 0, 0, 0, vel_desired, 0, -9.81])
        
    return x_ref_temp

In [118]:
def get_joint_torques(f_x, f_y, f_z, theta1, theta2, theta3, theta4, theta5, phi, theta, psi):
    return np.array([[f_x*(0.41*sin(theta2)*sin(psi + theta1)*cos(theta3) + 0.4*sin(theta2)*sin(psi + theta1)*cos(theta3 + theta4) + 0.04*sin(theta2)*sin(psi + theta1)*cos(theta3 + theta4 + theta5) - 0.41*sin(theta3)*cos(psi + theta1) - 0.4*sin(theta3 + theta4)*cos(psi + theta1) - 0.04*sin(theta3 + theta4 + theta5)*cos(psi + theta1))*cos(theta) + f_y*((sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*(0.41*sin(theta1)*sin(theta3) + 0.4*sin(theta1)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta3 + theta4 + theta5) + 0.41*sin(theta2)*cos(theta1)*cos(theta3) + 0.4*sin(theta2)*cos(theta1)*cos(theta3 + theta4) + 0.04*sin(theta2)*cos(theta1)*cos(theta3 + theta4 + theta5)) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*(0.41*sin(theta1)*sin(theta2)*cos(theta3) + 0.4*sin(theta1)*sin(theta2)*cos(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*cos(theta3 + theta4 + theta5) - 0.41*sin(theta3)*cos(theta1) - 0.4*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta3 + theta4 + theta5)*cos(theta1))) + f_z*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*(0.41*sin(theta1)*sin(theta2)*cos(theta3) + 0.4*sin(theta1)*sin(theta2)*cos(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*cos(theta3 + theta4 + theta5) - 0.41*sin(theta3)*cos(theta1) - 0.4*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta3 + theta4 + theta5)*cos(theta1)) - (sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(0.41*sin(theta1)*sin(theta3) + 0.4*sin(theta1)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta3 + theta4 + theta5) + 0.41*sin(theta2)*cos(theta1)*cos(theta3) + 0.4*sin(theta2)*cos(theta1)*cos(theta3 + theta4) + 0.04*sin(theta2)*cos(theta1)*cos(theta3 + theta4 + theta5)))], [f_x*((0.41*sin(theta2)*cos(theta3) + 0.4*sin(theta2)*cos(theta3 + theta4) + 0.04*sin(theta2)*cos(theta3 + theta4 + theta5))*sin(theta) + (0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*sin(psi)*sin(theta1)*cos(theta) - (0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*cos(psi)*cos(theta)*cos(theta1)) - f_y*(-(sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*(0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*sin(theta1) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*(0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*cos(theta1) + (0.41*sin(theta2)*cos(theta3) + 0.4*sin(theta2)*cos(theta3 + theta4) + 0.04*sin(theta2)*cos(theta3 + theta4 + theta5))*sin(phi)*cos(theta)) - f_z*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*(0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*cos(theta1) + (sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(0.41*cos(theta2)*cos(theta3) + 0.4*cos(theta2)*cos(theta3 + theta4) + 0.04*cos(theta2)*cos(theta3 + theta4 + theta5))*sin(theta1) - (0.41*sin(theta2)*cos(theta3) + 0.4*sin(theta2)*cos(theta3 + theta4) + 0.04*sin(theta2)*cos(theta3 + theta4 + theta5))*cos(phi)*cos(theta))], [-f_x*(-0.41*sin(theta)*sin(theta3)*cos(theta2) - 0.4*sin(theta)*sin(theta3 + theta4)*cos(theta2) - 0.04*sin(theta)*sin(theta3 + theta4 + theta5)*cos(theta2) - 0.41*sin(theta2)*sin(theta3)*cos(theta)*cos(psi + theta1) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta)*cos(psi + theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta)*cos(psi + theta1) + 0.41*sin(psi + theta1)*cos(theta)*cos(theta3) + 0.4*sin(psi + theta1)*cos(theta)*cos(theta3 + theta4) + 0.04*sin(psi + theta1)*cos(theta)*cos(theta3 + theta4 + theta5)) - f_y*((sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*(0.41*sin(theta1)*sin(theta2)*sin(theta3) + 0.4*sin(theta1)*sin(theta2)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + 0.41*cos(theta1)*cos(theta3) + 0.4*cos(theta1)*cos(theta3 + theta4) + 0.04*cos(theta1)*cos(theta3 + theta4 + theta5)) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*(0.41*sin(theta1)*cos(theta3) + 0.4*sin(theta1)*cos(theta3 + theta4) + 0.04*sin(theta1)*cos(theta3 + theta4 + theta5) - 0.41*sin(theta2)*sin(theta3)*cos(theta1) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1)) - (-0.41*sin(theta3) - 0.4*sin(theta3 + theta4) - 0.04*sin(theta3 + theta4 + theta5))*sin(phi)*cos(theta)*cos(theta2)) - f_z*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*(0.41*sin(theta1)*cos(theta3) + 0.4*sin(theta1)*cos(theta3 + theta4) + 0.04*sin(theta1)*cos(theta3 + theta4 + theta5) - 0.41*sin(theta2)*sin(theta3)*cos(theta1) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1)) - (sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(0.41*sin(theta1)*sin(theta2)*sin(theta3) + 0.4*sin(theta1)*sin(theta2)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + 0.41*cos(theta1)*cos(theta3) + 0.4*cos(theta1)*cos(theta3 + theta4) + 0.04*cos(theta1)*cos(theta3 + theta4 + theta5)) + (-0.41*sin(theta3) - 0.4*sin(theta3 + theta4) - 0.04*sin(theta3 + theta4 + theta5))*cos(phi)*cos(theta)*cos(theta2))], [-f_x*(-0.4*sin(theta)*sin(theta3 + theta4)*cos(theta2) - 0.04*sin(theta)*sin(theta3 + theta4 + theta5)*cos(theta2) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta)*cos(psi + theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta)*cos(psi + theta1) + 0.4*sin(psi + theta1)*cos(theta)*cos(theta3 + theta4) + 0.04*sin(psi + theta1)*cos(theta)*cos(theta3 + theta4 + theta5)) - f_y*((sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*(0.4*sin(theta1)*sin(theta2)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + 0.4*cos(theta1)*cos(theta3 + theta4) + 0.04*cos(theta1)*cos(theta3 + theta4 + theta5)) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*(0.4*sin(theta1)*cos(theta3 + theta4) + 0.04*sin(theta1)*cos(theta3 + theta4 + theta5) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1)) - (-0.4*sin(theta3 + theta4) - 0.04*sin(theta3 + theta4 + theta5))*sin(phi)*cos(theta)*cos(theta2)) - f_z*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*(0.4*sin(theta1)*cos(theta3 + theta4) + 0.04*sin(theta1)*cos(theta3 + theta4 + theta5) - 0.4*sin(theta2)*sin(theta3 + theta4)*cos(theta1) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1)) - (sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(0.4*sin(theta1)*sin(theta2)*sin(theta3 + theta4) + 0.04*sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + 0.4*cos(theta1)*cos(theta3 + theta4) + 0.04*cos(theta1)*cos(theta3 + theta4 + theta5)) + (-0.4*sin(theta3 + theta4) - 0.04*sin(theta3 + theta4 + theta5))*cos(phi)*cos(theta)*cos(theta2))], [-f_x*(-0.04*sin(theta)*sin(theta3 + theta4 + theta5)*cos(theta2) - 0.04*sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta)*cos(psi + theta1) + 0.04*sin(psi + theta1)*cos(theta)*cos(theta3 + theta4 + theta5)) - f_y*(-0.04*(-sin(theta1)*cos(theta3 + theta4 + theta5) + sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1))*(sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi)) + 0.04*(sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*(sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + cos(theta1)*cos(theta3 + theta4 + theta5)) + 0.04*sin(phi)*sin(theta3 + theta4 + theta5)*cos(theta)*cos(theta2)) - f_z*(-0.04*(sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*(-sin(theta1)*cos(theta3 + theta4 + theta5) + sin(theta2)*sin(theta3 + theta4 + theta5)*cos(theta1)) - 0.04*(sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(sin(theta1)*sin(theta2)*sin(theta3 + theta4 + theta5) + cos(theta1)*cos(theta3 + theta4 + theta5)) - 0.04*sin(theta3 + theta4 + theta5)*cos(phi)*cos(theta)*cos(theta2))]])

In [ ]:
# MPC for full point mass model #######################################################
from casadi import *

dt = 1/30 # [s] (sampling time interval)
N = 30 # Prediction horizon Length

f_min = -1000 # Minimum Force 
f_max = 1000 # Maximum Force. Replace with max force in x, z, and z

phi = SX.sym('phi') # Orientation Euler Angle 1
theta = SX.sym('theta') # Orientation Euler Angle 2
psi = SX.sym('psi') # Orientatoin Euler Angle 3

omega_x = SX.sym('omega_x')
omega_y = SX.sym('omega_y')
omega_z = SX.sym('omega_z')

p_x = SX.sym('p_x') # position X
p_y = SX.sym('p_y') # position Y
p_z = SX.sym('p_z') # position Z

v_x = SX.sym('v_x') # velocity X
v_y = SX.sym('v_y') # velocity Y
v_z = SX.sym('v_z') # velocity Z

g_constant = SX.sym('g') # gravity state / constant (is augmented to allow state space form), might be seperated again for readability

states = [phi, theta, psi, p_x, p_y, p_z, omega_x, omega_y, omega_z, v_x, v_y, v_z, g_constant]
n = len(states)

f_x_l = SX.sym('f_x_l') # Force in X on left foot
f_y_l = SX.sym('f_y_l') # Force in Y on left foot
f_z_l = SX.sym('f_z_l') # Force in Z on left foot

f_x_r = SX.sym('f_x_r') # Force in X on right foot
f_y_r = SX.sym('f_y_r') # Force in Y on right foot
f_z_r = SX.sym('f_z_r') # Force in Z on right foot

controls = [f_x_l, f_y_l, f_z_l, f_x_r, f_y_r, f_z_r]
m = len(controls)

U = SX.sym('U', m, N) # Control action matrix (that will be determined by the NLP solver)
X = SX.sym('X', n, N + 1) # State Matrix (that will also be determined by the NLP solver due to the chosen multiple-shooting method). The + 1 is for x0
P_rows = n
P_cols = 1 + N + n * N + m * N + N * m # last N* m for D matrix at each time step (used for contact constraint, passing D for every timestep allows to inform MPC about future contacts), REDUCE DIMENSIONALITY BY STATING CONTACT AS 1 OR 0!!!
print("P_cols:", P_cols)
P = SX.sym('P', P_rows, P_cols) # Parameter Matrix

#print("U shape after init:", U.shape)
#print("X shape after init:", X.shape)

objective_function = 0 # expression for the objective function

# lbx = lower bounds on optimization variable(s)
# ubx = upper bounds on optimization variable(s)
# lbg = lower bounds on constraints vector (should also be a vector), for equality constraints, just set lbg=ubg=k, where k is the constraint value
# ubg = upper bounds on constraints vector (should also be a vector)

lbx = []
ubx = []
lbg = []
ubg = []

g = [] # Constraint vector (both equality and inequality)

g += [X[:, 0] - P[:, 0]] # equality constraint x0 (chosen by solver) - x0 (input as parameter by user) = 0

lbg += [0] * n # n zeroes for equality constraints. These exist to ensure solver and actual initial state are the same.
ubg += [0] * n # n zeroes for equality constraints. These exist to ensure solver and actual initial state are the same.

#print("lbg length after initial state constraint:", len(lbg))

#Proven to be reasonbly stable:
#Q = np.diag([10000,10000,10000,50000,50000,50000,1000,1000,1000,10000,10000,10000,0])
#R = np.diag([0.007, 0.007, 0.007, 0.007, 0.007, 0.007])

Q = np.diag([10000,10000,10000,50000,50000,50000,1000,1000,1000,10000,10000,10000,0])

R = np.diag([0.007, 0.007, 0.007, 0.007, 0.007, 0.007])

t = 0

# state is phi, theta, psi, p_x, p_y, p_z, omega_x, omega_y, omega_z, v_x, v_y, v_z, gravity constant

x_t = [0, 0, 0, 0, 0, 1.48, 0, 0, 0, 0, 0, 0, -9.81]

state_history = []
reference_state_history = []
optimal_state_history = []
optimal_control_history = []
control_history = []
r_left_history = []
r_right_history = []
t_history = []

state_error_history = []
contact_matrix_history = []

state_full_control_applied_history = []
first_control_history = []

torque_history = []

orientation_predicted_history = []
vel_predicted_history = []

contact_history = []

#x_ref = get_integrated_position_trajectory(x_t[5])

#x_ref = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -9.81]

#x_ref = np.tile(np.arrxtay(x_ref).reshape(n, 1), N)

x_ref = get_integrated_position_trajectory(x_t[4], 0.2)

m_value = 30 # kg

P_param = np.zeros((P_rows, P_cols))

P_param[:, 0] = np.array(x_t).copy()

swing_left = True
swing_right = False

for i in range(N):
    #print("i:", i)
    
    state = X[:, i] # extract state at current time step
    control = U[:, i] # extract control input at current time step
    
    # Add the symbolic cost for each time step to the objective function based on different contact sets
    U_ref = P[m:m+m, 1 + N + n*N + m*N + (i*m)]
    
    objective_function = objective_function + (state - P[:, i+1]).T @ Q @ (state - P[:, i+1]) + (control - U_ref).T @ R @ (control - U_ref)
    # This is the equality constraint required for multiple shooting, so that the solver respects the system dynamics
    next_state = X[:, i+1]
    A_d_t = P[:, 1+N+(i*n):1+N+(i*n)+n]
    B_d_t = P[:, 1 + N + n*N +(i*m):1 + N + n*N + (i*m)+m]
    
    #print("A_d_t.shape:", A_d_t.shape)
    #print("B_d_t.shape:", B_d_t.shape)
    next_state_simulation = A_d_t @ state + B_d_t @ control
    #print("next_state_simulation:", next_state_simulation)
    #print("next state simulation type:", type(next_state_simulation))
    
    g += [next_state - next_state_simulation]
    
    lbg += [0] * n # n zeroes for equality constraints enforcing system dynamcis on every state
    ubg += [0] * n # n zeroes for equality constraints enforcing system dynamcis on every state
        

adjust_friction_constraints = True
mu = 0.5

adjust_contact_constraints_equality = True
adjust_contact_constraints_bounds = False

if adjust_contact_constraints_equality:

    for i in range(N): # Contact constraints.
        #if i % m == 0:
        g += [P[:m, 1 + N + n*N + m*N + (i*m):1 + N + n*N + m*N + (i*m) + m] @ U[:, i]]
        lbg += [0] * m
        ubg += [0] * m

for i in range(N): # Friction cone constraints. Seperate loops to have a nice structure in constraints

    if adjust_friction_constraints:
        if i % m == 0:
            
            f_x_left = U[0, i]
            f_y_left = U[1, i]
            f_z_left = U[2, i]
            
            f_x_right = U[3, i]
            f_y_right = U[4, i]
            f_z_right = U[5, i]
            
            # -mu * f_z < f_x
            g += [-mu * f_z_left - f_x_left]
            lbg += [-inf]
            ubg += [inf]
            
            # f_x < mu * f_z
            g += [f_x_left - mu * f_z_left]
            lbg += [-inf]
            ubg += [inf]
            
            # -mu * f_z < f_y
            g += [-mu * f_z_left - f_y_left]
            lbg += [-inf]
            ubg += [inf]
            
            # f_y < mu * f_z
            g += [f_y_left - mu * f_z_left]
            lbg += [-inf]
            ubg += [inf]
            
            # -mu * f_z < f_x
            g += [-mu * f_z_right - f_x_right]
            lbg += [-inf]
            ubg += [inf]
            
            # f_x < mu * f_z
            g += [f_x_right - mu * f_z_right]
            lbg += [-inf]
            ubg += [inf]
            
            # -mu * f_z < f_y
            g += [-mu * f_z_right - f_y_right]
            lbg += [-inf]
            ubg += [inf]
            
            # f_y < mu * f_z
            g += [f_y_right - mu * f_z_right]
            lbg += [-inf]
            ubg += [inf]  
            
optimization_variables = [X.reshape((n * (N+1), 1)), U.reshape((m * N, 1))]

nlp = {'x':vertcat(*optimization_variables), 'f':objective_function, 'g':vertcat(*g), 'p':P}

opts = {}
opts["print_time"] = 0
#opts["expand"] = False
opts['ipopt'] = {"max_iter":50, "print_level":0, "acceptable_tol":1e-7, "acceptable_obj_change_tol":1e-5}

solver = nlpsol('solver', 'ipopt', nlp, opts);

# Initialization values for solver. These should be filled with at least a linear interpolation between initial and desired state to improve performance.
# They are updated every iteration.

U_t = np.zeros((m, N))
X_t = np.matlib.repmat(np.array(x_t).reshape(n,1), N+1, 1)#.reshape(n*(N+1), 1) # np.tile(np.array(x_t).reshape(n, 1), N+1)

#print("U_t shape after init:", U_t.shape)
#print("X_t shape after init:", X_t.shape)

simulation_time = 20 # [s]
    
iterations = 0

r_left_contact = True
r_right_contact = True

I_body = np.array([[0.836, 0., 0.],
                    [0., 1.2288, 0.],
                    [0., 0., 1.4843]]) # Inertia in the body frame (meaning around CoM of torso).

Ixx = I_body[0, 0]
Ixy = I_body[0, 1]
Ixz = I_body[0, 2]

Iyx = I_body[1, 0]
Iyy = I_body[1, 1]
Iyz = I_body[1, 2]

Izx = I_body[2, 0]
Izy = I_body[2, 1]
Izz = I_body[2, 2]

for i in range(0, n * (N+1) + m * N):
    
    # TODO: Add state constraints
    if i <= n * (N+1) - 1: # Loop is still inside the state decision variable region in the vector.
        lbx += [-inf]
        ubx += [inf]
    else:
        
        lbx += [f_min]
        ubx += [f_max]
        
if adjust_contact_constraints_bounds:

    for i in range(0, m * N):
        if i % m == 0 or i == n * (N+1):

            index = n * (N+1)

            if not r_left_contact:
                lbx[index] = ubx[index] = 0.
                lbx[index + 1] = ubx[index + 1] = 0.
                lbx[index + 2] = ubx[index + 2] = 0.

            if not r_right_contact:
                lbx[index + 3] = ubx[index + 3] = 0.
                lbx[index + 4] = ubx[index + 4] = 0.
                lbx[index + 5] = ubx[index + 5] = 0.
            
print("Minimum possible Force in X and Y direction:", mu * f_min)
print("Maximum possible Force in X and Y direction:", mu * f_max)

use_C_code = False

solver.generate_dependencies('nlp.c')

if use_C_code:

    print("Compiling C Code for solver...")

    solver.generate_dependencies('nlp.c')
    import os
    os.system("gcc -O2 -fPIC -shared nlp.c -o nlp.so")
    solver = nlpsol("solver", "ipopt", "nlp.so", opts)

    print("Finished compiling.")

#X_t = X_t.reshape((n * (N+1), 1))
U_t = U_t.reshape((m * N, 1))

print("Entering main MPC loop...")

foot_behind_left = not swing_left # this is because it will be inverted on the first contact switch
foot_behind_right = not swing_right

pos_y_desired = 0
vel_y_desired = 0.2

psi_desired = 0
omega_z_desired = 0

step_length = 0.2

r_y_left = r_y_right = 0

contact_swap_interval = 15

D_vector = np.zeros(((simulation_time * int(1/dt)*m) * 2, m)) # * 2 because I'm lazy, the MPC just needs future contacts, even at the last iteration so 
counter = 0
for i in range(D_vector.shape[0]):
    if i % m == 0:
        if counter % contact_swap_interval == 0:
            swing_left = not swing_left
            swing_right = not swing_right
            D_t = np.array([[int(swing_left == True), 0, 0, 0, 0, 0],
                            [0, int(swing_left == True), 0, 0, 0, 0],
                            [0, 0, int(swing_left == True), 0, 0, 0],
                            [0, 0, 0, int(swing_right == True), 0, 0],
                            [0, 0, 0, 0, int(swing_right == True), 0],
                            [0, 0, 0, 0, 0, int(swing_right == True)]]) # Swing = no contact (and thus 1 any force == 0 must mean force is 0)
        
        D_vector[i:i + m, :] = D_t
        counter += 1
        
x_t = np.array(x_t).reshape(n,1)

while t < simulation_time:
    # In C++: Update I_world based on rotation matrix angles -> Update A_d and B_d based on I world and r_l, r_r changes -> Pass A_d and B_d as parameters
    
    D_t = D_vector[iterations*m:iterations*m + m, :].copy()
    P_param[:m, 1 + N + n*N + m*N:] = D_vector[iterations*m:iterations*m + N*m, :].T.copy()
    contact_matrix_history.append(P_param[:m, 1 + N + n*N + m*N:].copy())
    
    for k in range(N):
        #print("D_k in P_Param:\n", P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m])
        if P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][0, 0] == 1 and P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][3, 3] == 1: # No feet in contact
            P_param[m:m+m, 1 + N + n*N + m*N + (k*m)] = np.array([0, 0, 0, 0, 0, 0]).copy()
            #print("No feet in contact")
        elif P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][0, 0] == 1 and  P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][3, 3] == 0: # Right foot in contact
            P_param[m:m+m, 1 + N + n*N + m*N + (k*m)] = np.array([0, 0, 0, 0, 0, m_value * 9.81]).copy()
            #print("Right foot in contact")
        elif  P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][0, 0] == 0 and  P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][3, 3] == 1: # Left foot in contact
            P_param[m:m+m, 1 + N + n*N + m*N + (k*m)] = np.array([0, 0, m_value * 9.81, 0, 0, 0]).copy()
            #print("Left foot in contact")
        if  P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][0, 0] == 0 and  P_param[:m, 1 + N + n*N + m*N + k*m: 1 + N + n*N + m*N + k*m + m][3, 3] == 0: # Both feet in contact
            P_param[m:m+m, 1 + N + n*N + m*N + (k*m)] = np.array([0, 0, (m_value * 9.81) / 2, 0, 0, (m_value * 9.81) / 2]).copy()
            #print("Both feet in contact")
        #print("U_ref_k in P_Param:\n", P_param[m:m+m, 1 + N + n*N + m*N + (k*m)])
    
    contact_history.append([P_param[:m, 1 + N + n*N + m*N:1 + N + n*N + m*N+m][0,0].copy(), P_param[:m, 1 + N + n*N + m*N:1 + N + n*N + m*N+m][3,3]].copy())
    if iterations % (contact_swap_interval * 2) == 0:

        if foot_behind_left:
            r_y_left = -step_length
        else:
            r_y_left = step_length
        foot_behind_left = not foot_behind_left

        if foot_behind_right:
            r_y_right = -step_length
        else:
            r_y_right = step_length
        foot_behind_right = not foot_behind_right
                
        # Location of the force vector being applied by the left foot.
        r_x_left = 0.15
        #r_y_left = 0
        r_z_left = -x_t[5]

        # Location of the force vector being applied by the right foot.
        r_x_right = -0.15
        #r_y_right = 0
        r_z_right = -x_t[5]
        
    x_ref = np.zeros((n, N))
    
    pos_y_temp = pos_y_desired
    psi_temp = psi_desired
    
    for i in range(N):
        pos_y_temp += vel_y_desired * dt
        psi_temp += omega_z_desired * dt
        x_ref[:, i] = np.array([0, 0, psi_temp, 0, pos_y_temp, 1.48, omega_z_desired, 0, 0, 0, vel_y_desired, 0, -9.81]).reshape(n)
        
    pos_y_desired += vel_y_desired * dt
    psi_desired += omega_z_desired * dt

    P_param[:, 1:N+1] = x_ref.copy()

    reference_state_history.append(x_ref[:, 0].reshape(n,1).copy()) # or -1
    
    state_error_history.append(x_ref[:, 0].copy().reshape(n,1) - x_t.copy())
    
    #print(x_t - X_t[n:n+n])
    
    for i in range(N):
        if i < N-1:
            phi_t = X_t[n*(i+1)+0]
            theta_t = X_t[n*(i+1)+1]
            psi_t = X_t[n*(i+1)+2]
            
            vel_x_t = X_t[n*(i+1)+9]
            vel_y_t = X_t[n*(i+1)+10]
            vel_z_t = X_t[n*(i+1)+11]
            
            pos_x_t = X_t[n*(i+1)+3]
            pos_y_t = X_t[n*(i+1)+4]
            pos_y_t_next = X_t[n*(i+2)+4]
            pos_z_t = X_t[n*(i+1)+5]
        else:
            phi_t = X_t[n*(N-1)]
            theta_t = X_t[n*(N-1)+1]
            psi_t = X_t[n*(N-1)+2]
            
            vel_x_t = X_t[n*(N-1)+9]
            vel_y_t = X_t[n*(N-1)+10]
            vel_z_t = X_t[n*(N-1)+11]
            
            pos_x_t = X_t[n*(N-1)+3]
            pos_y_t = X_t[n*(N-1)+4]
            pos_y_t_next = X_t[n*N + 4]
            pos_z_t = X_t[n*(N-1)+5]
            
        if i == 0:
            phi_t = x_t[0]
            theta_t = x_t[1]
            psi_t = x_t[2]

            vel_x_t = x_t[9]
            vel_y_t = x_t[10]
            vel_z_t = x_t[11]

            pos_x_t = x_t[3]
            pos_y_t = x_t[4]
            pos_z_t = x_t[5]
                        
        #phi_t = x_ref[0, i]
        #theta_t = x_ref[1, i]
        #psi_t = x_ref[2, i]
        
        #r_y_left -= (pos_y_t_next - pos_y_t)
        #r_y_right -= (pos_y_t_next - pos_y_t)
        
        orientation_predicted_history.append([phi_t, theta_t, psi_t])
        
        vel_predicted_history.append([vel_x_t, vel_y_t, vel_z_t])

        I_world = np.array([[(Ixx*cos(psi_t) + Iyx*sin(psi_t))*cos(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*sin(psi_t), -(Ixx*cos(psi_t) + Iyx*sin(psi_t))*sin(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*cos(psi_t), Ixz*cos(psi_t) + Iyz*sin(psi_t)], [(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*cos(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*sin(psi_t), -(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*sin(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*cos(psi_t), -Ixz*sin(psi_t) + Iyz*cos(psi_t)], [Ixy*sin(psi_t) + Izx*cos(psi_t), Ixy*cos(psi_t) - Izx*sin(psi_t), Izz]])

        # Skew symmetric versions for the 3x1 foot position vector resembling the matrix version of the cross product of two vectors. This is needed for the matrix form.
        r_left_skew_symmetric = np.array([[0, -r_z_left, r_y_left],
                                        [r_z_left, 0, -r_x_left],
                                        [-r_y_left, r_x_left, 0]])

        r_right_skew_symmetric = np.array([[0, -r_z_right, r_y_right],
                                        [r_z_right, 0, -r_x_right],
                                        [-r_y_right, r_x_right, 0]])

        A_c = np.array([[0, 0, 0, 0, 0, 0, math.cos(psi_t), math.sin(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, -math.sin(psi_t), math.cos(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

        B_c = np.block([[0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [I_world @ r_left_skew_symmetric, I_world @ r_right_skew_symmetric],
                        [1/m_value, 0, 0, 1/m_value, 0, 0],
                        [0, 1/m_value, 0, 0, 1/m_value, 0],
                        [0, 0, 1/m_value, 0, 0, 1/m_value],
                        [0, 0, 0, 0, 0, 0]])

        A_d, B_d = discretize_ss(A_c, B_c, dt)
        
        P_param[:, 1 + N + (i*n):1 + N + (i*n)+n] = A_d.copy()
        P_param[:, 1 + N + n * N + (i*m):1 + N + n * N + (i*m)+m] = B_d.copy()
    
    x0_solver = vertcat(*[X_t, U_t])
    sol = solver(x0=x0_solver, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg, p=DM(P_param))
    
    t_history.append(t)
    
    u_t = sol['x'][n * (N+1):n * (N+1) + m]
    
    x_temp_t = sol['x'][:n]
    x_temp_vector = np.zeros((n*N, 1))
    
    for k in range(N):
        u_temp_t = sol['x'][n*(N+1) + (k*m): n*(N+1) + (k*m) + m]
        
        phi_t = x_temp_t[0]
        theta_t = x_temp_t[1]
        psi_t = x_temp_t[2]

        I_world = np.array([[(Ixx*cos(psi_t) + Iyx*sin(psi_t))*cos(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*sin(psi_t), -(Ixx*cos(psi_t) + Iyx*sin(psi_t))*sin(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*cos(psi_t), Ixz*cos(psi_t) + Iyz*sin(psi_t)], [(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*cos(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*sin(psi_t), -(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*sin(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*cos(psi_t), -Ixz*sin(psi_t) + Iyz*cos(psi_t)], [Ixy*sin(psi_t) + Izx*cos(psi_t), Ixy*cos(psi_t) - Izx*sin(psi_t), Izz]])

        #r has to be updated here in future as well!!!

        A_c = np.array([[0, 0, 0, 0, 0, 0, math.cos(psi_t), math.sin(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, -math.sin(psi_t), math.cos(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

        B_c = np.block([[0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [I_world @ r_left_skew_symmetric, I_world @ r_right_skew_symmetric],
                        [1/m_value, 0, 0, 1/m_value, 0, 0],
                        [0, 1/m_value, 0, 0, 1/m_value, 0],
                        [0, 0, 1/m_value, 0, 0, 1/m_value],
                        [0, 0, 0, 0, 0, 0]])

        A_d, B_d = discretize_ss(A_c, B_c, dt)

        x_temp_t = A_d @ np.array(x_temp_t).reshape(n,1) + B_d @ np.array(u_t).reshape(m,1)
        x_temp_vector[k*n:k*n+n] = x_temp_t
        
    state_full_control_applied_history.append(x_temp_vector.copy())
    
    tau_l = get_joint_torques(u_t[0], u_t[1], u_t[2], math.radians(0), math.radians(0), math.radians(15), math.radians(-10), math.radians(10), x_t[0], x_t[1], x_t[2])
    tau_r = get_joint_torques(u_t[3], u_t[4], u_t[5], math.radians(0), math.radians(0), math.radians(15), math.radians(-10), math.radians(10), x_t[0], x_t[1], x_t[2])
    torque_history.append([tau_l.copy(), tau_r.copy()])
        
    phi_t = x_t[0]
    theta_t = x_t[1]
    psi_t = x_t[2]
    
    I_world = np.array([[(Ixx*cos(psi_t) + Iyx*sin(psi_t))*cos(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*sin(psi_t), -(Ixx*cos(psi_t) + Iyx*sin(psi_t))*sin(psi_t) + (Ixy*cos(psi_t) + Iyy*sin(psi_t))*cos(psi_t), Ixz*cos(psi_t) + Iyz*sin(psi_t)], [(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*cos(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*sin(psi_t), -(-Ixx*sin(psi_t) + Iyx*cos(psi_t))*sin(psi_t) + (-Ixy*sin(psi_t) + Iyy*cos(psi_t))*cos(psi_t), -Ixz*sin(psi_t) + Iyz*cos(psi_t)], [Ixy*sin(psi_t) + Izx*cos(psi_t), Ixy*cos(psi_t) - Izx*sin(psi_t), Izz]])
    
    #r has to be updated here in future as well!!!
    
    A_c = np.array([[0, 0, 0, 0, 0, 0, math.cos(psi_t), math.sin(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, -math.sin(psi_t), math.cos(psi_t), 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],

                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

    B_c = np.block([[0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [I_world @ r_left_skew_symmetric, I_world @ r_right_skew_symmetric],
                        [1/m_value, 0, 0, 1/m_value, 0, 0],
                        [0, 1/m_value, 0, 0, 1/m_value, 0],
                        [0, 0, 1/m_value, 0, 0, 1/m_value],
                        [0, 0, 0, 0, 0, 0]])
    
    A_d, B_d = discretize_ss(A_c, B_c, dt)
    
    x_t = A_d @ np.array(x_t).reshape(n,1) + B_d @ np.array(u_t).reshape(m,1)
    P_param[:, 0] = x_t.reshape(n).copy()
    
    X_t[:-n] = sol['x'][n:n * (N+1)]
    X_t[-n:] = sol['x'][-n - m * N: n * (N+1)]
    
    U_t[:-m] = sol['x'][m + n * (N+1):]
    U_t[-m:] = sol['x'][-m:]
    
    optimal_state_history.append(X_t.copy())
    optimal_control_history.append(U_t.copy())
    
    first_control_history.append(sol['x'][n*(N+1):n*(N+1)+m])
    
    #print(X_t[n:n+n] - x_t)
    
    state_history.append(x_t)
    control_history.append(np.array(u_t).reshape(m,1))
    r_left_history.append([r_x_left, r_y_left, r_z_left])
    r_right_history.append([r_x_right, r_y_right, r_z_right])

    t += dt
    iterations += 1
    
print("Final MPC iterations:", iterations)
print("Steady state error:\n", (x_ref[: ,-1].reshape(n,1) - x_t))


plt.rcParams['figure.figsize'] = [10, 4] #inches
plt.rcParams.update({'figure.max_open_warning': 0})

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[3] for x in state_history],label="pos_x")
ax.plot(t_history, [x[4] for x in state_history],label="pos_y")
ax.plot(t_history, [x[5] for x in state_history],label="pos_z")

t_history_interval = int(len(t_history) / int(len(t_history) / N)) # I know this is ugly, but who cares :)

ax.plot(t_history, [x[3][0] for x in reference_state_history], label="pos_x_ref", linestyle='--')
ax.plot(t_history, [x[4][0] for x in reference_state_history], label="pos_y_ref", linestyle='--')
ax.plot(t_history, [x[5][0] for x in reference_state_history], label="pos_z_ref", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[0] for x in contact_history], label="contact_left")
ax.plot(t_history, [x[1] for x in contact_history], label="contact_right")

ax.plot(t_history, [x[1] for x in r_left_history], label="r_y_left")
ax.plot(t_history, [x[2] for x in r_left_history], label="r_z_left")

ax.plot(t_history, [x[1] for x in r_right_history], label="r_y_right")
ax.plot(t_history, [x[2] for x in r_right_history], label="r_z_right")
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[9] for x in state_history],label="vel_x")
ax.plot(t_history, [x[10] for x in state_history],label="vel_y")
ax.plot(t_history, [x[11] for x in state_history],label="vel_z")

ax.plot(t_history, [x[9][0] for x in reference_state_history], label="vel_x_ref", linestyle='--')
ax.plot(t_history, [x[10][0] for x in reference_state_history], label="vel_y_ref", linestyle='--')
ax.plot(t_history, [x[11][0] for x in reference_state_history], label="vel_z_ref", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[0] for x in state_history],label="phi")
ax.plot(t_history, [x[1] for x in state_history],label="theta")
ax.plot(t_history, [x[2] for x in state_history],label="psi")

ax.plot(t_history, [x[0][0] for x in reference_state_history], label="phi_ref", linestyle='--')
ax.plot(t_history, [x[1][0] for x in reference_state_history], label="theta_ref", linestyle='--')
ax.plot(t_history, [x[2][0] for x in reference_state_history], label="psi_ref", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[6] for x in state_history],label="omega_x")
ax.plot(t_history, [x[7] for x in state_history],label="omega_y")
ax.plot(t_history, [x[8] for x in state_history],label="omega_z")

ax.plot(t_history, [x[6][0] for x in reference_state_history], label="omega_x_ref", linestyle='--')
ax.plot(t_history, [x[7][0] for x in reference_state_history], label="omega_y_ref", linestyle='--')
ax.plot(t_history, [x[8][0] for x in reference_state_history], label="omega_z_ref", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[0] for x in control_history], label="f_x_left")
ax.plot(t_history, [x[1] for x in control_history], label="f_y_left")
ax.plot(t_history, [x[2] for x in control_history], label="f_z_left")

# no minus needed because f_min is already negative, maybe change that.
ax.plot(t_history, [-mu * f[2] for f in control_history], label="lower_limit_x", linestyle='--')
ax.plot(t_history, [-mu * f[2] for f in control_history], label="lower_limit_y", linestyle='--')
ax.plot(t_history, np.ones(len(t_history)) * f_min, label="lower_limit_z", linestyle='--')

ax.plot(t_history, [mu * f[2] for f in control_history], label="upper_limit_x", linestyle='--')
ax.plot(t_history, [mu * f[2] for f in control_history], label="upper_limit_y", linestyle='--')
ax.plot(t_history, np.ones(len(t_history)) * f_max, label="upper_limit_z", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[3] for x in control_history], label="f_x_right")
ax.plot(t_history, [x[4] for x in control_history], label="f_y_right")
ax.plot(t_history, [x[5] for x in control_history], label="f_z_right")

# no minus needed because f_min is already negative, maybe change that.

ax.plot(t_history, [-mu * f[5] for f in control_history], label="lower_limit_x", linestyle='--')
ax.plot(t_history, [-mu * f[5] for f in control_history], label="lower_limit_y", linestyle='--')
ax.plot(t_history, np.ones(len(t_history)) * f_min, label="lower_limit_z", linestyle='--')

ax.plot(t_history, [mu * f[5] for f in control_history], label="upper_limit_x", linestyle='--')
ax.plot(t_history, [mu * f[5] for f in control_history], label="upper_limit_y", linestyle='--')
ax.plot(t_history, np.ones(len(t_history)) * f_max, label="upper_limit_z", linestyle='--')
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[0][0] for x in torque_history], label="tau_1_l")
ax.plot(t_history, [x[0][1] for x in torque_history], label="tau_2_l")
ax.plot(t_history, [x[0][2] for x in torque_history], label="tau_3_l")
ax.plot(t_history, [x[0][3] for x in torque_history], label="tau_4_l")
ax.plot(t_history, [x[0][4] for x in torque_history], label="tau_5_l")
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[1][0] for x in torque_history], label="tau_1_r")
ax.plot(t_history, [x[1][1] for x in torque_history], label="tau_2_r")
ax.plot(t_history, [x[1][2] for x in torque_history], label="tau_3_r")
ax.plot(t_history, [x[1][3] for x in torque_history], label="tau_4_r")
ax.plot(t_history, [x[1][4] for x in torque_history], label="tau_5_r")
plt.legend()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t_history, [x[0] for x in state_error_history], label="phi_error")
ax.plot(t_history, [x[1] for x in state_error_history], label="theta_error")
ax.plot(t_history, [x[2] for x in state_error_history], label="psi_error")
ax.plot(t_history, [x[3] for x in state_error_history], label="pos_x_error")
ax.plot(t_history, [x[4] for x in state_error_history], label="pos_y_error")
ax.plot(t_history, [x[5] for x in state_error_history], label="pos_z_error")
ax.plot(t_history, [x[6] for x in state_error_history], label="vel_x_error")
ax.plot(t_history, [x[7] for x in state_error_history], label="vel_y_error")
ax.plot(t_history, [x[8] for x in state_error_history], label="vel_z_error")
ax.plot(t_history, [x[9] for x in state_error_history], label="omega_x_error")
ax.plot(t_history, [x[10] for x in state_error_history], label="omega_y_error")
ax.plot(t_history, [x[11] for x in state_error_history], label="omega_z_error")
plt.legend()

plot_optimal_state_history = True

plot_interval = 5

if plot_optimal_state_history:
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(t_history, [x[0] for x in state_history], 'k', label="phi_actual")

    for y in range(0, len(optimal_state_history), plot_interval):
        ax.plot(tuple(numeric_range(y*dt, y*dt+(N+1)*dt, dt)), optimal_state_history[y][0::n], label="phi_mpc")
        if y == 0:
            plt.legend()
            
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(t_history, [x[5] for x in state_history], 'k', label="pos_z_actual")

    for y in range(0, len(optimal_state_history), plot_interval):
        ax.plot(tuple(numeric_range(y*dt, y*dt+(N+1)*dt, dt)), optimal_state_history[y][5::n], label="pos_z_mpc")
        if y == 0:
            plt.legend()
            
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(t_history, [x[2] for x in control_history], 'k', label="u_t")
    #ax.plot(t_history, [first_control[2] for first_control in first_control_history], label="u_mpc")
    plt.legend()

    for y in range(0, len(optimal_control_history), plot_interval):
        #ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][0::m], label="u_t_1")
        #ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][1::m], label="u_t_2")
        ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][2::m],'r', label="u_t_3")
        #ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][3::m], label="u_t_4")
        #ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][4::m], label="u_t_5")
        #ax.plot(tuple(numeric_range(y*dt, y*dt+N*dt, dt)), optimal_control_history[y][5::m], label="u_t_6")
        
        if y == 0:
            plt.legend()

P_cols: 781
Minimum possible Force in X and Y direction: -500.0
Maximum possible Force in X and Y direction: 500.0
Entering main MPC loop...


In [10]:
solver.generate_dependencies('nlp.c')
import os
os.system("gcc -fPIC -shared -O3 nlp.c -o nlp.so")

opts["print_time"] = 1
solver = nlpsol("solver", "ipopt", "nlp.so", opts)

x_t = [0.3, 0.3, 0.3, 0, 0, 1, 0, 0, 0, 0, 0, 0, -9.81]
x_ref = [0, 0, 0, 0.5, 0.5, 0.5, 0, 0, 0, 0, 0, 0, -9.81]

U_t = np.zeros((m, N))
X_t = np.tile(np.array(x_t).reshape(n, 1), N+1).reshape(n, N+1)

x0_solver = vertcat(*[X_t.reshape((n * (N+1), 1)), U_t.reshape((m * N, 1))])

sol = solver(x0=x0_solver, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg, p=P_param)

TypeError: 'Symbol' object cannot be interpreted as an integer

In [9]:
m = Symbol("m") # Total mass of the Robot including Legs (combined into point mass)
g_constant = Symbol("g") # Gravity acceleration constant used in g-vector
g = Matrix([0, 0, -g_constant])

In [4]:
# Symbols for inertia tensor of the point mass

Ixx, Ixy, Ixz = symbols("Ixx Ixy Ixz")
Iyx, Iyy, Iyz = symbols("Iyx Iyy Iyz")
Izx, Izy, Izz = symbols("Izx Izy Izz")

# Inertia Tensor of the Robot (as a point mass) in body frame coordinates.
# This should include Torso and Legs in zero configuration

I_body = Matrix([[Ixx, Ixy, Ixz],
                [Iyx, Iyy, Iyz],
                [Izx, Ixy, Izz]])

I_body

Matrix([
[Ixx, Ixy, Ixz],
[Iyx, Iyy, Iyz],
[Izx, Ixy, Izz]])

In [5]:
f_x_l, f_y_l, f_z_l = symbols("f_x_l f_y_l f_z_l") # Symbols for left reaction force vector

f_l = Matrix([f_x_l, f_y_l, f_z_l])  # 3x1 Reaction force vector of the left leg

f_l

Matrix([
[f_x_l],
[f_y_l],
[f_z_l]])

In [6]:
f_x_r, f_y_r, f_z_r = symbols("f_x_r f_y_r f_z_r") # Symbols for right reaction force vector

f_r = Matrix([f_x_r, f_y_r, f_z_r])  # 3x1 Reaction force vector of the right leg

f_r

Matrix([
[f_x_r],
[f_y_r],
[f_z_r]])

In [7]:
# Explanation for Rigid Body Dynamics of a point mass:
# https://phys.libretexts.org/Bookshelves/University_Physics/Book%3A_University_Physics_(OpenStax)/Map%3A_University_Physics_I_-_Mechanics%2C_Sound%2C_Oscillations%2C_and_Waves_(OpenStax)/10%3A_Fixed-Axis_Rotation__Introduction/10.08%3A_Newton%E2%80%99s_Second_Law_for_Rotation

In [8]:
phi, theta, psi = symbols("phi theta psi") # Euler angles of the point mass used for orientation
phi_dot, theta_dot, psi_dot = symbols("phidot thetadot psidot") # Derivatives of Euler angles of the point mass

omega_x, omega_y, omega_z = symbols("omega_x omega_y omega_z") # Angular velocity of point mass in X-Y-Z

omega = Matrix([omega_x, omega_y, omega_z]) # 3x1 vector describing angular velocity of the point mass

omega

Matrix([
[omega_x],
[omega_y],
[omega_z]])

In [9]:
# Converting 3x1 vector to 3x3 skew symmetric matrix based on: 
# https://math.stackexchange.com/questions/2248413/skew-symmetric-matrix-of-vector
# See also: https://en.wikipedia.org/wiki/Skew-symmetric_matrix
Image(url="https://cdn.discordapp.com/attachments/680811067848655093/689769467890368541/unknown.png")

In [10]:
omega_skew_symmetric = Matrix([[0, -omega.row(2)[0], omega.row(1)[0]],
                               [omega.row(2)[0], 0, -omega.row(0)[0]],
                               [-omega.row(1)[0], omega.row(0)[0], 0]])
omega_skew_symmetric

Matrix([
[       0, -omega_z,  omega_y],
[ omega_z,        0, -omega_x],
[-omega_y,  omega_x,        0]])

In [11]:
r_x_l, r_y_l, r_z_l = symbols("r_x_l r_y_l r_z_l")
r_l = Matrix([r_x_l, r_y_l, r_z_l]) # Location of left foot ground reaction force

r_l

Matrix([
[r_x_l],
[r_y_l],
[r_z_l]])

In [12]:
r_l_skew_symmetric = Matrix([[0, -r_l.row(2)[0], r_l.row(1)[0]],
                             [r_l.row(2)[0], 0, -r_l.row(0)[0]],
                             [-r_l.row(1)[0], r_l.row(0)[0], 0]])
r_l_skew_symmetric

Matrix([
[     0, -r_z_l,  r_y_l],
[ r_z_l,      0, -r_x_l],
[-r_y_l,  r_x_l,      0]])

In [13]:
r_x_r, r_y_r, r_z_r = symbols("r_x_r r_y_r r_z_r")
r_r = Matrix([r_x_r, r_y_r, r_z_r]) # Location of right foot ground reaction force

r_r

Matrix([
[r_x_r],
[r_y_r],
[r_z_r]])

In [14]:
r_r_skew_symmetric = Matrix([[0, -r_r.row(2)[0], r_r.row(1)[0]],
                             [r_r.row(2)[0], 0, -r_r.row(0)[0]],
                             [-r_r.row(1)[0], r_r.row(0)[0], 0]])
r_r_skew_symmetric

Matrix([
[     0, -r_z_r,  r_y_r],
[ r_z_r,      0, -r_x_r],
[-r_y_r,  r_x_r,      0]])

In [21]:
R_z = Matrix([[cos(psi), -sin(psi), 0],
              [sin(psi), cos(psi), 0],
              [0, 0, 1]])

R_y = Matrix([[cos(theta), 0, sin(theta)],
              [0, 1, 0],
              [-sin(theta), 0, cos(theta)]])

R_x = Matrix([[1, 0, 0],
              [0, cos(phi), -sin(phi)],
              [0, sin(phi), cos(phi)]])

R = R_z * R_y * R_x # Rotation matrix from Body frame to World frame based on Z-Y-X Rotation order

R = R.inv() # Invert matrix as the angular velocity of the point mass is omega_world = R * omega_body ("omega" in the notebook)
R = simplify(R.subs({phi:0, theta:0})) # Omit roll and pitch of the point mass because they should be very small.

R # will be identical to R_z

Matrix([
[ cos(psi), sin(psi), 0],
[-sin(psi), cos(psi), 0],
[        0,        0, 1]])

In [22]:
I_world = R * I_body * R.T # Transform / Rotate body inertia tensor into the world coordinate frame

I_world

Matrix([
[  (Ixx*cos(psi) + Iyx*sin(psi))*cos(psi) + (Ixy*cos(psi) + Iyy*sin(psi))*sin(psi),   -(Ixx*cos(psi) + Iyx*sin(psi))*sin(psi) + (Ixy*cos(psi) + Iyy*sin(psi))*cos(psi),  Ixz*cos(psi) + Iyz*sin(psi)],
[(-Ixx*sin(psi) + Iyx*cos(psi))*cos(psi) + (-Ixy*sin(psi) + Iyy*cos(psi))*sin(psi), -(-Ixx*sin(psi) + Iyx*cos(psi))*sin(psi) + (-Ixy*sin(psi) + Iyy*cos(psi))*cos(psi), -Ixz*sin(psi) + Iyz*cos(psi)],
[                                                      Ixy*sin(psi) + Izx*cos(psi),                                                        Ixy*cos(psi) - Izx*sin(psi),                          Izz]])

In [23]:
str(I_world)

'Matrix([[(Ixx*cos(psi) + Iyx*sin(psi))*cos(psi) + (Ixy*cos(psi) + Iyy*sin(psi))*sin(psi), -(Ixx*cos(psi) + Iyx*sin(psi))*sin(psi) + (Ixy*cos(psi) + Iyy*sin(psi))*cos(psi), Ixz*cos(psi) + Iyz*sin(psi)], [(-Ixx*sin(psi) + Iyx*cos(psi))*cos(psi) + (-Ixy*sin(psi) + Iyy*cos(psi))*sin(psi), -(-Ixx*sin(psi) + Iyx*cos(psi))*sin(psi) + (-Ixy*sin(psi) + Iyy*cos(psi))*cos(psi), -Ixz*sin(psi) + Iyz*cos(psi)], [Ixy*sin(psi) + Izx*cos(psi), Ixy*cos(psi) - Izx*sin(psi), Izz]])'